In [1]:
import torch
import torch.nn as nn

In [271]:
class TestNet(nn.Module):
    def __init__(self):
        super(TestNet, self).__init__()
        
        hidden_size = 2
        #self.lstm = nn.LSTM(2, hidden_size, 1)
        self.lstm = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 16), 
            nn.ReLU(),
            #nn.BatchNorm1d(1),
            nn.Linear(16, 16)
        )
        self.linear = nn.Linear(16, 1)
        
        def init_weights(m):
            if isinstance(m, nn.Linear):
                torch.nn.init.xavier_normal_(m.weight)
                m.bias.data.fill_(0.01)
                
        self.lstm.apply(init_weights)
        self.linear.apply(init_weights)
        
    def forward(self, x):
        out_total = torch.zeros(size=(1, 1, 16))
        
        for i in range(x.shape[0]):
            inp = x[i].unsqueeze(0).unsqueeze(0)
            out = self.lstm(inp)
            out_total += out
            
        out_total = out_total / x.shape[0]
            
        print(out_total)
        #out_total = nn.Sigmoid()(out_total)
        #print(out_total)
        out = self.linear(out_total)
        print(out)
            
        return nn.Sigmoid()(out)

In [272]:
tn = TestNet()

x = torch.arange(10.).reshape((5,2))
print(x)

out = tn(x)
print(out)
print()

x = torch.arange(2000., 2010.).reshape((5,2))
print(x)

out = tn(x)
print(out)

tensor([[0., 1.],
        [2., 3.],
        [4., 5.],
        [6., 7.],
        [8., 9.]])
tensor([[[-0.5965,  0.2560, -0.1156, -1.3574, -0.3386, -0.0248, -0.8628,
          -0.1483,  2.2814, -1.6396,  0.2900,  0.3806,  0.0604, -0.6295,
          -0.2692, -0.7012]]], grad_fn=<DivBackward0>)
tensor([[[-0.7319]]], grad_fn=<AddBackward0>)
tensor([[[0.3248]]], grad_fn=<SigmoidBackward0>)

tensor([[2000., 2001.],
        [2002., 2003.],
        [2004., 2005.],
        [2006., 2007.],
        [2008., 2009.]])
tensor([[[-212.7543,  100.1948,  -55.2785, -596.1447, -140.3803,   30.3395,
          -338.2869,  -72.1009, 1067.6324, -761.0759,  121.8217,  212.2877,
            21.2918, -225.2295,  -57.8166, -341.9248]]],
       grad_fn=<DivBackward0>)
tensor([[[-318.6244]]], grad_fn=<AddBackward0>)
tensor([[[0.]]], grad_fn=<SigmoidBackward0>)


In [ ]:
optimizer = torch.optim.Adam(tn.parameters(), lr=0.1)

for i in range(100):
    high = bool(torch.randint(2, (1,)).item())
    
    if high: 
        x = torch.randint(2000, 2050, (5, 2), dtype=torch.float32)
        #x = torch.ones(size=(5,2))
        y = torch.tensor([1.])
    else:
        x = torch.randint(0, 20, (5, 2), dtype=torch.float32)
        #x = torch.zeros(size=(5,2))
        y = torch.tensor([0.])
        
    y = y.unsqueeze(0).unsqueeze(0)
        
    out = tn(x)
    
    optimizer.zero_grad()
    loss = nn.BCELoss()(out, y)
    loss.backward()
    optimizer.step()

    print(out, high)
    print(loss)
    print()